# Hierarchical Linear Model

## Hands-on material

Notebook ini berisikan perintah-pertintah dan dokumentasi dalam melakukan analisis _Hierarchical Linear Model_ yaitu sebuah rumpun model yang memiliki struktur data berjenjang

## A. Persiapan

### 1. Setting up working environment

In [3]:
set more off
version 12

In [4]:
cd "D:\Documents\OneDrive - Institut Teknologi Bandung\Documents\Teaching\SP6015\Mgg#10"

D:\Documents\OneDrive - Institut Teknologi Bandung\Documents\Teaching\SP6015\Mgg
> #10


### 2. Loading the dataset

Syarat utama dari HLM adalah bahwa struktur datanya bersifat bertingkat: variable lvl-1 bersarang dalam variabel lvl-2 dan bersarang dalam variabel lvl-3, dst. Kita akan mengawali dengan membuka dataset yang telah bersuktrut untuk mendemostrasikan bentuk struktur data yang akan digunalan dalam analisis HLM.

In [5]:
use hsb.dta, clear

In [6]:
describe


Contains data from hsb.dta
  obs:         7,185                          
 vars:             8                          27 Oct 2021 21:52
 size:       431,100                          
--------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
--------------------------------------------------------------------------------
schid           str4    %4s                   School ID
minority        double  %9.0g                 Minority Student- 0=white and
                                                1=non-white students
female          double  %9.0g                 Female student-coded 0 for male
                                                and 1 for female
ses             double  %9.0g                 A measure of student
                                                socio-economic status
mathach         double  %9.0g                 Student's performance o

### High School and Beyond

Data ini merupakan sub-sampel dari Survey High School and Beyond (1982) yang juga digunakan pada buku Hierarchical Linear Models oleh Raudenbush and Bryk. file dataset ini adalah hsb.dta yang berisikan 7185 siswa tingkat sekolah menengah atas di USA yang "bersarang"/nested di 160 sekolah. Dataset ini berisikan variable-variabel:
- schid: kode sekolah
- minority: siswa adalah termasuk minoritas (0=kulit putih, 1= non-kulit putih)
- female: siswa perempuan (0=laki-laki, 1= perempuan)
- ses: index/score status sosial ekonomi dari siswa
- mathach: nilai siswa pada test matematika
- size: jumlah siswa di sekolah
- schtype: tipe sekolah (1= sekolah swasta, 0 = sekolah negeri)
- meanses: rata-rata skor SES untuk setiap sekolah

Variable-variabel diatas terdiri dari 2 dataset. Dataset pertama adalah dataset untuk level individu siswa (hsb_student.dta) atau yang akan disebut seterusnya sebagai Level-1 dan dataset yang kedua adalah (hsb_school.dta), yang selanjutnya akan disebut sebagai Level-2. Dataset hsb_student.dta berisikan informasi tentang 7185 siswa, sedangkan hsb_school.dta berisikan informasi tentang 160 sekolah, dimana seluruh sampel siswa pada dataset hsb_student.dta bersekolah. Kedua dataset tersebut akan disatukan (merged-up) menjadi 1 dataset utuh.

Selanjutnya kita akan membuka dataset level 1 dan level 2 secara terpisah dan kemudian disatukan (merge) menjadi 1 dataset berstruktur hierarkis.

#### Buka dataset level-1

In [7]:
use "hsb_student.dta", clear

In [8]:
describe


Contains data from hsb_student.dta
  obs:         7,185                          
 vars:             6                          27 Oct 2021 22:09
 size:       287,400                          
--------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
--------------------------------------------------------------------------------
schid           str4    %4s                   School ID
pid             float   %9.0g                 Studednt ID
minority        double  %9.0g                 Minority Student- 0=white and
                                                1=non-white students
female          double  %9.0g                 Female student-coded 0 for male
                                                and 1 for female
ses             double  %9.0g                 A measure of student
                                                socio-economic status
mat

In [9]:
sum


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
       schid |          0
         pid |      7,185    24.50814    15.20242          1         67
    minority |      7,185     .274739    .4464137          0          1
      female |      7,185    .5281837    .4992398          0          1
         ses |      7,185    .0001434    .7793552     -3.758      2.692
-------------+---------------------------------------------------------
     mathach |      7,185    12.74785    6.878246     -2.832     24.993


#### Buka dataset level-2

In [10]:
use "hsb_school.dta", clear

In [11]:
describe


Contains data from hsb_school.dta
  obs:           160                          
 vars:             4                          27 Oct 2021 22:11
 size:         4,480                          
--------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
--------------------------------------------------------------------------------
schid           str4    %4s                   School ID
size            double  %9.0g                 Number of students in the school
schtype         double  %9.0g                 Private school -coded 0 for public
                                                and 1 for private schools
meanses         double  %9.0g                 the average SES for each school in
                                                the data set
--------------------------------------------------------------------------------
Sorted by: 


In [12]:
sum 


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
       schid |          0
        size |        160    1097.825    629.5064        100       2713
     schtype |        160       .4375    .4976359          0          1
     meanses |        160   -.0001875    .4139731     -1.188       .831


#### Merging the dataset level-1 ke dataset level-2

In [13]:
merge 1:m schid using "hsb_student.dta",  nogen


    Result                           # of obs.
    -----------------------------------------
    not matched                             0
    matched                             7,185  
    -----------------------------------------


### 3. Eksplorasi Data

#### a. Statistika deskriptif

In [14]:
sum


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
       schid |          0
        size |      7,185    1056.862    604.1725        100       2713
     schtype |      7,185    .4931106    .4999873          0          1
     meanses |      7,185    .0061385    .4135539     -1.188       .831
         pid |      7,185    24.50814    15.20242          1         67
-------------+---------------------------------------------------------
    minority |      7,185     .274739    .4464137          0          1
      female |      7,185    .5281837    .4992398          0          1
         ses |      7,185    .0001434    .7793552     -3.758      2.692
     mathach |      7,185    12.74785    6.878246     -2.832     24.993


#### b. Informasi tentang cluster

In [15]:
sum mathach

scalar om = r(mean)

bysort schid: gen sn = _N     // school n

by schid: gen first = _n == 1 // mark first obs in school

sum sn if first



    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
     mathach |      7,185    12.74785    6.878246     -2.832     24.993





    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
          sn |        160    44.90625    11.85489         14         67


#### c. Visualisasi data

In [ ]:
twoway (scatter mathach ses if schtype ==1, mcolor(blue) msize(small) msymbol(circle_hollow) ///
        mlwidth(vthin)) (scatter mathach ses if schtype ==0, mcolor(green) msize(vsmall) msymbol(square_hollow) ///
                         mlwidth(vvvthin)), ytitle(Student's Math Score) legend(order(1 "Catholic" 2 "Public"))


## Model Hierarchical Linear Model:



### Model 1: One-way ANOVA
Model paling sederhana yang melakukan partisi variance dari variabel dependent diantara level utnuk memberikan baseline untuk "improvement of fit" dari estimasi.

__Pertanyaan penelitian__ yang relevan: _Bagaimana variasi dalam skor test matematika diantara sekolah?_

### Model 2: Regresi Means-as-Outcomes
Spesifikasi model tanpa variable Level-1, dengan variable Level 2 memprediksi/menjelaskan group-means

__Pertanyaan penelitian__ yang relevan: _Bagaimana karakteristik sekolah menentukan variasi rata-rata nilai antar sekolah?_

### Model 3: Random-Coefficients Regression
Spesifikasi model Level-1 sederhana dengan memasukkan variable independent Level-1 dengan partisi variance

__Pertanyaan penelitian__ yang relevan: _Bagaimana karakteristik siswa berkaitan dengan skor test matematik?_

### Model 4: Intercepts- & Slopes-as-Outcomes
Spesifikasi model yang memperhatikan perbedaan pada koefisien di Level-1 dengan variabel Level-2

__Pertanyaan penelitian__ yang relevan: _Bagaimana pengaruh karakteristik siswa bervariasi diantara sekolah?_

### Model 1: One-way ANOVA dengan Random Effects: Empty Model

Pada model yang paling sederhana adalah model yang ekivalen dengan model ove-way ANOVA dengan random effects yang ditunjukkan pada persamaan berikut:
$$ 
Y_{ij}=\beta_{0j} + r_{ij}  
$$

Dengan persamaan diatas, dapat diasumsikan bahwa setiap error pada level-1 diatas adalah terdistribusi normal, dnegan nilai rata-data error sama dengan 0, dan variance konstan pada level-1, $ \sigma^{2} $. Pada dataset kit maka $i$ = 1, 2, ..., $ n_j $ adalah siswa di sekolah $j$, dan $j$=1,2,..., 160 sekolah. Model ini menujukka nilai rata pada test matematik  disetiap sekolah, yaitu $\beta_{0j}$.

Di Level-2 (tingkat sekolah), setiap nilai rata-rata skor matematika  $\beta_{0j}$ adalah fungsi dari nilai rata-rata keseluruhan (grand-mean) $\gamma_{00}$, dan random error $\mu_{0j}$:

$$ 
\beta_{0j}= \gamma_{00}+ u_{0j}  
$$

Dengan asumsi $u_{0j} ~ N(0,\tau_{00})$. $\tau_{00}$ adalah variansi di tingkat sekolah. 
Dari kedua persamaan diatas maka diperoleh model kombinasi atau yang dikenal dengan "mixed-model" dengan fixed-effect $\gamma_{00}$ dan random-effects $ u_{0j}$ dan $ r_{ij}$:

$$ 
Y_{ij}=\gamma_{00}+ u_{0j} + r_{ij}  
$$

In [16]:
mixed mathach || schid:, variance reml


Performing EM optimization: 

Performing gradient-based optimization: 

Iteration 0:   log restricted-likelihood = -23558.397  
Iteration 1:   log restricted-likelihood = -23558.397  

Computing standard errors:

Mixed-effects REML regression                   Number of obs     =      7,185
Group variable: schid                           Number of groups  =        160

                                                Obs per group:
                                                              min =         14
                                                              avg =       44.9
                                                              max =         67

                                                Wald chi2(0)      =          .
Log restricted-likelihood = -23558.397          Prob > chi2       =          .

------------------------------------------------------------------------------
     mathach |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
---------

### Hasil Estimasi

Dari hasil diatas, koefisien intercept atau $\tau_{00}$ adalah rata-rata dari nilai rata-rata sekolah (grand mean).

Di tingkat sekolah $\tau_{00}$ adalah variance dari "true" nilai rata-rata sekolah, $\beta_{0j}$, disekitar nilai grand mean $\gamma_{00}$. Estimasi variability dari rata-rata sekolah ditunjukkan pada nilai $var(cons)$ adalah:
$$
\hat{Var}(\beta_{0j})=\hat{Var}(u_{0j})=\hat{\tau}_{00}=8.61
$$

Untuk menilai magnitude dari variasi diantara sekolah dari rata-rata nilai matematika (outcome), dapat dihitung rentang nilai yang mungkin dari nilai rata-rata sekolah. Dalam asumsi normalitas, 95% dari nilai rata-rata sekolah berada dalam rentang:
$$
\hat{\tau}_{00}\pm 1.96 \sqrt{\hat{\tau}_{00}}
$$
yaitu:
$$
12.64 \pm 1.96 \sqrt{8.61}= (6.89,18.39)
$$

Dari hasil diatas diperoleh hasil bahwa intraclass correlation (ICC) $\rho$ = 0.180, yang dapat diartikan bahwa kurang labih 18% variasi pada skor matematika ditentukan oleh lingkungan sekolah.

Intraclass correlation $\hat\rho$, dapat dihitung dengan persamaan berikut:
$$
\hat\rho = \frac{\hat\tau_{00}}{\hat\tau_{00}+\hat\sigma^{2}} 
$$
yaitu:
$$
\hat\rho = \frac{8.614081}{8.614081+39.14832}= 0.18035
$$

Atau dengan post-estimation commad $ estat $ $icc$


In [17]:
estat icc


Intraclass correlation

------------------------------------------------------------------------------
                       Level |        ICC   Std. Err.     [95% Conf. Interval]
-----------------------------+------------------------------------------------
                       schid |   .1803528   .0187219      .1465168    .2199886
------------------------------------------------------------------------------


### Model 2: Regresi dengan Means-as-Outcomes

#### Model
Persamaan tingkat siswa tetap tidak berubah bahwa nilai matematika siswa dipandang bervariasi disekitar nilai rata-rata sekolahnya:
$$ 
Y_{ij}=\beta_{0j} + r_{ij}  
$$
Namun sekarang persamaan $ \beta_{0j}= \gamma_{00}+ u_{0j} $ dikembangkan sehingga setiap nilai rata-rata sekolah diprediksikan oleh variabel sekolah yaitu rata-rata kondisi sosial ekonomi siswa $MEANSES$ di sekolah:
$$ 
\beta_{0j}= \gamma_{00}+\gamma_{01}MEANSES+u_{0j}
$$
Dimana $\gamma_{00}$ adalah intercept, $\gamma_{01}$ adalah effect dari MEANSES terhadap $\beta_{0j}$ dan $u_{0j}$ diasumsikan terdistribusi normal dengan mean = 0, dan variance $\tau_{00}$.

Sekarang nilai $u_{0j}$ dan $\tau_{00}$ memiliki makna yang berbeda. kalau sebelumnya $u_{0j}$ adalah simpangan dari rata-rata sekolah $j$ dari grand mean, sekarang $u_{0j}$ merupakan residual $\beta_{0j}$ - $\gamma_{00}$ - $\gamma_{01}MEANSES$, dan $\tau_{00}$ adalah conditional variance $Var(\beta_{0j}|MEANSES)$ atau variance tingkat sekolah setelah mengontrol MEAN SES.

Sehingga persamaan $Y_{ij}=\beta_{0j} + r_{ij}  $ sekarang menjadi:
$$
Y_{ij}=\gamma_{00} + \gamma_{01}MEANSES + u_{0j} + r_{ij}  
$$
Dengan fixed effects $\gamma_{00}$,$\gamma_{01}$, dan random effects $u_{0j}$, $r_{ij}$

In [18]:
mixed mathach meanses || schid: , variance reml


Performing EM optimization: 

Performing gradient-based optimization: 

Iteration 0:   log restricted-likelihood = -23480.642  
Iteration 1:   log restricted-likelihood = -23480.642  

Computing standard errors:

Mixed-effects REML regression                   Number of obs     =      7,185
Group variable: schid                           Number of groups  =        160

                                                Obs per group:
                                                              min =         14
                                                              avg =       44.9
                                                              max =         67

                                                Wald chi2(1)      =     263.15
Log restricted-likelihood = -23480.642          Prob > chi2       =     0.0000

------------------------------------------------------------------------------
     mathach |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
---------

### Hasil Estimasi

#### Fixed effects
Dari hasil tersebut maka terdapat dua fixed-effects dari rata-rata nilai sekolah (school mean) yaitu parameter $\gamma_{00}$ (cons) dan $\gamma_{01}$ (meanses). Dari sini terlihat bahwa efek dari rata-rata sosial ekonomi sekolah meanses adalah signifikan ($\hat{\gamma}_{01}$ = 5.86 dengan nilai $Z$ = 16.22)

#### Variance Component

Variance dari residual antar sekolah $\hat{\tau}_{00}$ =2.64 jauh lebih kecil dari  $\hat{\tau}_{00}$ =8.61 pada model sebelumnya (empty model). Rentang nilai yang mungkin dari rata-rata sekolah dengan seluruh sekolah memiliki rata-rata $meanses$ = 0 adalah:

$$
\hat{\tau}_{00}\pm 1.96 \sqrt{\hat{\tau}_{00}} = 12.64\pm1.96 \sqrt{2.64}= (9.47, 15.83)
$$

Meskipun rentang ini masih cukup lebar, namun masih lebih kecil dari rentang nilai ketika nilai $meanses$ belum di-kontrol.

Dengan membandingkan $\tau_{00}$ dari kedua model, kita dapat menghitung sebuah index _proportions reduction in variance_ atau _"variance explained"_ pada Level-2 atau tingkat sekolah:

Proportion of variance explained in $\beta_{0j}$

$$
=\frac{\hat{\tau}_{00}(random ANOVA)-\hat{\tau}_{00}(MEAN SES)}{\hat{\tau}_{00}(random ANOVA)}= \frac{(8.61-2.64)}{8.61}=0.69
$$

Artinya, sekitar 69% dari "true" variance antar sekolah pada skor matematika ditentukan oleh $MEAN SES$
Setelah mengontrol effect dari rata-rata SES (MEAN SES), maka korelasai antar pasangan nilai di pada sekolah yang sama, yang awalnya adalah 0.18, sekarang berkurang menjadi
$$
\frac{2.64}{2.64+39.16}=.06
$$
__Kesimpulannya:__ 

Kita dapat mengetahui bahwa model _means-as-outcomes_ ini rata-rata kondisi SES di sekolah memiliki asosiasi positif yang signifikan terhadap rata-rata nilai matematika siswa. Meski demikian, meskipun telah mengontrol MEAN SES, sekolah masih bervariasi dalam hal mencapaian nilai matematikanya.

## Model 3: Model Random Koefisien

Pada tahap ini dilakukan analisis hubungan antara status sosial ekonomi siswa SES dengan skor tes matematiknya di setiap 160 sekolah. Dengan kata lain, setiap sekolah seolah-olah memiliki persamaan regresi masing-masing lengkap dengan intercept dan slope/koefisien SES. Untuk itu kita menanyakan beberapa pertanyaan berikut:
- Bagaimana rata-rata dari ke 160 persamaan regresi (yaitu: Berapa nilai rata-rata _intercept_ dan _slope_/koefisien SES)
- Seberapa besar persamaan-persamaan regresi bervariasi dari satu sekolah ke sekolah lain? Dengan kata lain, sejauh mana _intercept_ bervariasi, dan seberapa jauh _slope_ SES bervariasi?
- Bagaiman korelasi antar _intercept_ dan _slopes SES? Apakah sekolah dengan nilai _intercept_ besar, yaitu memiliki rata-rata nilai matematika tinggi juga memiliki nilai _slope_ SES yang tinggi (korelasi antara SES dengan nilai skor)?

### Model
Model dasar diberikan oleh persamaan berikut:

$$
Y_{ij}=\beta_{0j}+\beta_{1j}X_{ij}+r_{ij}
$$

Pada persamaan diatas, interpretasi dari _intercept_ adalah nilai ekspektasi dari nilai $Y$ ketika nilai prediktor pada level individu (siswa) adalah = 0. Hal ini tentu saja perlu diperhatika  dengan seksama, karena pada beberapa kasus tidak terdaapt nilai prediktor=0  secara logis. Misalnya untuk variabel _age_ atau _SES_. Untuk variabel_age_, nilai _intercept_ menjadi nilai ekpektasi dari nilai skor matematika apabila _age_ dari individu $ij$ adalah 0. Tentu saja nilai _age_=0 tidak dapat diterima pada konteks ini disebabkan tidak ada seseorang yang berumur 0 yang sudah bersekolah. Untuk itu, perlu dilakukan proses rescaling-recentering. Pembahasan pilihan recentering akan dibahas pada section lain. Pada pembahasan ini, nilai prediktor di rescale /recenter pada rerata sekolah. Oleh karena itu persamaa regresi menjadi sbb:

$$
Y_{ij}=\beta_{0j}+\beta_{1j}(X_{ij}-\bar{X}_{.j})+r_{ij}
$$

Pada persamaan ini, nilai predictor pada tingkat siswa di recenter disekitar rerata sekolah (school-mean), maka estiamsi _intercept_ adalah nilai ekspektasi dari outcome ketika nilai prediktor sama dengan nilai rata-rata sekolah. 

Pada parameter diatas nilai $\beta_{0j}$ dan $\beta_{1j}$ dapat bervariasi antar sekolah pada model tingkat 2 sebagai fungsi dari _grand mean_ dan _random error_:

$$
\beta_{0j}=\gamma_{00}+u_{0j}
$$

$$
\beta_{1j}=\gamma_{10}+u_{1j}
$$

Komponen $ u_{0j} $ dan $ u_{1j} $ ini yang menjadikan model ini memperoleh sebutan sebagai "_Random-Coefficient_". Dimana:
- $\gamma_{00}$ adalah nilai rata-rata dari rerata sekolah dalam nilai skor test matematika dari keseluruhan populasi sekolah (_grand mean_)
- $\gamma_{10}$ adalah nilai rata-rata _slope_ regresi antara variabel SES dengan nilai skor matematika di antara sekolah-sekolah tersebut
- $u_{0j}$ adalah perubahan/berbedaan unik dari nilai _intercept_ atau grand mean_ dari sekolah $j$
- $u_{1j}$ adalah perubahan/perbedaan unik dari nilai _slope_ dari sekolah $j$

Apabila persamaan padal Level-1 dan Level-2 di substitusikan, maka akan diperoleh sebuah model campuran (_mixed model_)

$$
Y_{ij}=\gamma_{00}+\gamma_{10}(X_{ij}-\bar{X}_{.j})+u_{0j}+u_{1j}(X_{ij}-\bar{X}_{.j})+r_{ij}
$$

Untuk komponen _variance_, dapat ditentukan sbb:

$$
Var(u_{0j})=\tau_{00}
$$
$$
Var(u_{1j})=\tau_{11}
$$
dan kovariansi diantara keduanya adalah:
$$
Cov(u_{0j},u_{1j})=\tau_{01}
$$

Dimana:
$$
Var(u_{0j})=Var(\beta_{0j}-\gamma_{00})= Var(\beta_{0j})
$$
$$
Var(u_{1j})=Var(\beta_{1j}-\gamma_{10})= Var(\beta_{1j})
$$
Hal ini berarti bahwa model regresi _random-coefficient_ memberika estimasi untuk parameter variabelitias pada _random intercepts_ dan _slopes_.

In [19]:
gen grp_cent_ses = ses - meanses // Kita lakukan school-mean centering (group-centering)

In [20]:
sum  ses grp_cent_ses


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
         ses |      7,185    .0001434    .7793552     -3.758      2.692
grp_cent_ses |      7,185   -.0059951    .6605881     -3.657       2.85


In [21]:
mixed mathach grp_cent_ses || schid: grp_cent_ses, cov(un) reml // Jalankan mixed model


Performing EM optimization: 

Performing gradient-based optimization: 

Iteration 0:   log restricted-likelihood =  -23357.18  
Iteration 1:   log restricted-likelihood = -23357.118  
Iteration 2:   log restricted-likelihood = -23357.118  

Computing standard errors:

Mixed-effects REML regression                   Number of obs     =      7,185
Group variable: schid                           Number of groups  =        160

                                                Obs per group:
                                                              min =         14
                                                              avg =       44.9
                                                              max =         67

                                                Wald chi2(1)      =     292.40
Log restricted-likelihood = -23357.118          Prob > chi2       =     0.0000

------------------------------------------------------------------------------
     mathach |      Coef.   Std.

In [22]:
estat icc


Conditional intraclass correlation

------------------------------------------------------------------------------
                       Level |        ICC   Std. Err.     [95% Conf. Interval]
-----------------------------+------------------------------------------------
                       schid |   .1913031   .0194602      .1560252    .2323614
------------------------------------------------------------------------------
Note: ICC is conditional on zero values of random-effects covariates.


### Hasil Estimasi

#### Fixed effects:

Ada dua fixed effects yang ditunjukkan dengan koefisien _grp_cent_ses_ dan _cons_. Koefisien _cons_ adalah rata-rata keseluruhan dari nilai rerata nilai matematika sekolah, dan Koefisien grp_cent_ses adalah rata-rata slope dari SES-dan nilai matematika di sekolah

#### Komponen Variance-Covariance:
- _var( cons)_:
adalah estimasi variance dari rerata sekolah $\hat{\tau}_{00}$ = 8.68 dan signifikan. Ini diartikan bahwa terdapat perbedaan signifikan diantara rata-rata nilai matematika sekolah (Konsisten dengan hasil empty model)

- var(grp_ce~s): estimated variance of the slopes yaitu $\hat{\tau}_{11}$ = 0.69 dan signifikan. Ini diartikan bahwa hubungan antara SES dan nilai skor matematik di dalam sekolah bervariasi secara signifikan diantara semua populasi sekolah

Nilai yang mungkin untuk nilai rerata sekolah dan _slopes_ SES-skor matematika spesifik untuk tiap sekolah dapat dihitung dengan fomula berikut untuk rentang nilai yang mungkin pada tingkat kepercayaan 95%:

$$
\hat{\gamma}_{q0}\pm1.96\sqrt{\hat{\tau}_{qq}}
$$
Maka:
- Nilai rerata skor matematika untuk setiap sekolah adalah $12.64\pm1.96\sqrt{8.68}=(6.87, 18.41) $
- Nilai slope SES-skor matematika untuk tiap sekolah yang mungkin adalah $ 2.19\pm1.96\sqrt{0.69}=(0.56,3.81) $

Dengan membandingkan $\tau_{00}$ dari kedua model, kita dapat menghitung sebuah index _proportions reduction in variance_ atau _"variance explained"_ pada Level-2 atau tingkat sekolah:

_Proportion of variance explaiend_ in $\beta_{0j}$
$$
=\frac{\hat{\tau}_{00}(random ANOVA)-\hat{\tau}_{00}(SES)}{\hat{\tau}_{00}(random ANOVA)}= \frac{(39.15-36.70)}{39.15}=0.063
$$
Artinya, SES menentukan sekitar 6.3% terhadap variance pada tingkat siswa/individu  pada outcome. Ketika dibandingkan dengan models sebelumnya dimana MEAN SES menentukan sekitar 60% dari variansi antar sekolak pada outcome, maka jelas bahwa asosiasi antara dua variabel ini jauh lebih kuat di tingkat sekolah dibandiungkan pada tingkat individu siswa.


## Model 4: Model Intercept and Slopes as Outcomes


Setelah sebelumnya melakukan estimasi variabilitas dari persamaan regresi dari seluruh sekolah ( satu persamaan regresi untuk setiap sekolah), kini saatnya untuk membangun model eksplanatoris untuk menjelaskan variabilitas dari persamaan regresi tersebut. Secara spesifik, kita ingin memahami mengapa beberapa sekolah memiliki rerata nilai skor matematika lebih tinggi dibandingka sekolah-sekolah lain, dan menjelaskan mengapa pada beberapa sekolah asosiasi antara SES dengan skor matematika lebih kuat dibandingkan sekolah lain.

#### Model
Model pada tingkat siswa (level-1) masih tetap sama pada model sebelumnya:

$$
Y_{ij}=\beta_{0j}+\beta_{1j}(X_{ij}-\bar{X}_{.j})+r_{ij}
$$

Yang membedakan adalah, model pada tingkat sekolah (level-2) dikembangkan dengan memasukkan prediktor tingkat sekolah yaitu SCHTYPE dan MEAN SES. Dengan kata lain kita ingin memodelkan bahwa baik nilai rerata skor matematika maupun hubungan antara status SES siswa dengan nilai skor matematika di tingkat sekolah adalah ditentukan juga oleh konteks sekolah, yiatu status SES siswa rata-rata (MEAN SES) dan jenik sekolah (SCHTYPE). Maka persamaan regresi di tingkat sekolah adalah:
$$
\beta_{0j}=\gamma_{00}+\gamma_{01}(MEAN SES)_{j}+\gamma_{02}(SCHTYPE)_{j}+u_{0j} $$
$$
\beta_{10}=\gamma_{11}+\gamma_{01}(MEAN SES)_{j}+\gamma_{12}(SCHTYPE)_{j}+u_{1j}
$$

Dengan mengkombinasikan model pada tingkat sekolah (school-level/Level-2) dengan model pada tingkat individu/siswa (student-level/Level-1), maka akan diperoleh persamaan campuran berikut ini:

$$Y_{ij}=\gamma_{00}+\gamma_{01}(MEAN SES)_{j}+\gamma_{02}(SCHTYPE)_{j}$$
$$+\gamma_{10}(X_{ij}-\bar{X}_{.j})+\gamma_{11}(MEAN SES)_{j}(X_{ij}-\bar{X}_{.j})$$
$$+\gamma_{12}(SCHTYPE)(X_{ij}-\bar{X}_{.j})+u_{0j}+u_{1j}(X_{ij}-\bar{x}_{.j})+r_{ij}$$

Persamaan diatas menggambarkan bahwa outcome (skor nilai matematika individu) merupakan sebuah fungsi dari _overall intercept_ $(\gamma_{00})$, efek utama dari MEAN SES $(\gamma_{01})$, efek utama dari SCHTYPE $(\gamma{02})$, efek utama dari SES $(\gamma_{10})$, dan 2 buah efek interaksi antar tingkat (_cross level interaction_) yang terdiri dari SCHTYPE dengan status SES siswa $(\gamma_{12})$ dan MEAN SES dengan status SES siswa $()\gamma_{11}$, dengan sebuah komponen _random error_:
$$
u_{1j}(X_{ij}-\bar{X}_{.j})+r_{ij}
$$

Dari persamaan diatas, dapat diuraikan lebih lanjut beberapa jenis pertanyaan yang menjadi motivasi untuk analisis:
- Apakah MEAN SES dan SCHTYPE secara signifikan memprediksi intersep (estimasi parameter $\gamma_{01}$ dan $\gamma_{02}$)?
- Apakah MEAN SES dan SCHTYPE secara signifikan memprediksi kemiringan atau _slopes_ SES dengan skor matematika siswa di dalam tiap sekolah (_within-school_) (estimasi parameter $\gamma_{11}$ dan $\gamma_{12}$)?
- Seberapa besar variasi dari _intercept_ dan _slopes_ dapat dijelaskan denngan menggunakan SCHTYPE dan MEAN SES sebagai prediktor? (Estimasi komponen variansi $Var(u_{0j})=\tau_{00}$ dan $Var(u_{1j})=\tau_{11}$, dan membandingkannya dengan hasil estimasi yang diperoleh pada model _random coefficient_ sebelumnya)

In [24]:
mixed mathach meanses schtype grp_cent_ses /// 
    c.meanses#c.grp_cent_ses i.schtype#c.grp_cent_ses || schid: grp_cent_ses , reml cov(un)


Performing EM optimization: 

Performing gradient-based optimization: 

Iteration 0:   log restricted-likelihood = -23252.888  
Iteration 1:   log restricted-likelihood = -23251.837  
Iteration 2:   log restricted-likelihood = -23251.834  
Iteration 3:   log restricted-likelihood = -23251.834  

Computing standard errors:

Mixed-effects REML regression                   Number of obs     =      7,185
Group variable: schid                           Number of groups  =        160

                                                Obs per group:
                                                              min =         14
                                                              avg =       44.9
                                                              max =         67

                                                Wald chi2(5)      =     746.33
Log restricted-likelihood = -23251.834          Prob > chi2       =     0.0000

-------------------------------------------------------

### Hasil Estimasi:

#### Fixed Effect
Hasil diatas dapat kita interpretasikan sebagai berikut:
- Koefisien _meanses_ ($\hat{\gamma}_{01}$) menunjukkan bahwa MEAN SES berhubungan positif secara signifikas dengan rata-rata sekolah nilai skor matematika ($\hat{\gamma}_{01} = 5.34, z=14.46$)
- Koefisien _schtype_ ($\hat{\gamma}_{02}$) menunjukkan bahwa SCHTYPE, dalam hal ini sekolah swasta/sekolah katolik memiliki nilai rata-rata skor matematika dibandingkan sekolah negeri/publik ($\hat{\gamma}_{02}=1.22, z=3.97 $)
- Koefisien _grp_cent_ses_ ($\hat{\gamma}_{10}$) menunjukkan bahwa status SES siswa secara umum berkontribusi pada peningkatan skor matematika siswa sebesar 2.94 point untuk setiap peningkatan 1 point dalam status SES siswa  ($\hat{\gamma}_{10}=2.94, z=18.95 $).

Pada koefisien interaksi:
- Koefisien _c.meanses#c.grp_cent_ses_ ($\hat{\gamma}_{11}$) menunjukkan bahwa sekolah dengan MEAN SES yang tinggi cenderung memiliki nilai _slopes_ SES dengan skor matematika yang lebih besar, mengindikasikan bahmwa sekolah dengan nilai MEAN SES cenderung memiliki kekuatan hubungan antara SES dengan skor matematika yang lebih kuat secara positif dan signifikasn ($\hat{\gamma}_{11}=1.04, z= 3.48$).
- Koefisien _schtype#c.grp_cent_ses_ ($\hat{\gamma}_{12}$) menunjukkan bahwa sekolah swasta/katolik cenderung memiliki kekuatan hubungan (_slopes_) antara SES dengan skor matematika yang lebih rendah dibandingkan sekolah negeri/publik ($\hat{\gamma}_{12}=-1.64, z=-6.85$)

#### Komponen Variance-Covariance
Secara Subjektif, dapat kita bandingkan hasil estimasi komponen variance-covariancce antara model ini dengan model sebelumnya. Pada model ini terlihat bahwa rata-rata variance rata-rata sekolah ($u_0j$) dan variance slope ses-skor matematika ($u_1j$) adalah 2.38 dan 0.10, dibandingkan pada model _random coefficient_ yang memiliki nilai estimasi variance 8.68 dan 0.69

Seperti halnya sebelumnya, kita dapat menghitung proporsi reduksi variance-covariance yang terjelaskan oleh model  (_proportions reduction in variance-covariance_) pada koefisien $\beta_{qj}$
$$
=\frac{\hat{\tau}_{qq}(random regression)-\hat{\tau}_{qq}(fitted model)}{\hat{\tau}_{qq}(random regression)}
$$

$$
=\frac{8.68-2.38}{8.68}=0.73
$$

Hasil diatas menunjukkan bahwa dengan variabel level-2 MEAN SES dan SCHTYPE berhasil menjelaskan variasi pada estimasi parameter rata-rata sekolah dalam skor matematika sebesar 73% dari model sebelumnya (random coefficient). Demikian pula untuk variansi residual, variabel level-2 MEAN SES dan SCHTYPE menjelaskan variasi residual sebesar 54% ($\frac{0.69-0.1}{0.1}=0.54$)


In [25]:
estat ic


Akaike's information criterion and Bayesian information criterion

-----------------------------------------------------------------------------
       Model |        Obs  ll(null)  ll(model)      df         AIC        BIC
-------------+---------------------------------------------------------------
           . |      7,185         .  -23251.83      10    46523.67   46592.47
-----------------------------------------------------------------------------
               Note: N=Obs used in calculating BIC; see [R] BIC note.


## How to Decide Centering Independent Variables

Yang dimaksud dengan _"centering"_ dalam analisis HLM adalah merubah nilai sedemikian rupa untuk memperbaiki interpretasi model. Terdapat 3 jenis centering dalam analisis HLM:
- uncentered $(X_{ij})$ : variabel dipertahankan pada nilai aslinya. Interpretasinya bahwa zero = 0
- cluster/group-mean centering $(X_{ij}-\bar{X}_{.j})$ : Variabel dirubah sedemikian rupa sehingga menunjukkan perbedaan antara individu dengan groupnya. Interpretasi zero = group mean
- grand-mean centering $(X_{ij}-\bar{\bar{X}}_{..})$: variabel dirubah sedemikin rupa sehingga menunjukkan perbedaa individu dengan nilai rata-rata keseluruhan populasi. Interpretasinya zero = overall mean

Konsekuensi Centering:
- dapat merubah interpretasi dari intercept pada level 1
- dapar mempengaruhi estimasi dari slopes

In [26]:
gen ses_uncentered = ses //uncentered

egen gmses=mean(ses), by(schid) //group-mean centered
gen ses_group_ctrd = ses - gmses

egen grandmses = mean(gmses) //grand-mean centered
gen ses_grand_ctrd = ses - grandmses

sum ses_uncentered ses_group_ctrd ses_grand_ctrd








    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
ses_uncent~d |      7,185    .0001434    .7793552     -3.758      2.692
ses_group_~d |      7,185   -1.62e-10     .660588  -3.650741   2.856078
ses_grand_~d |      7,185    4.67e-10    .7793552  -3.758143   2.691857
